In [1]:
import dynalearn as dl
import h5py
import pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import tensorflow.keras as keras
from time import time
import tqdm
import os

In [2]:
# Graph parameters
N = 100
avgk = 5
graph_generator = dl.generators.ERGraph(N, avgk / N)

# Dynamics parameters
inf_prob = 0.04
rec_prob = 0.08
init_state = None
state_generator = dl.generators.SISDynamics(inf_prob, rec_prob, init_state)

# Data generator parameters
batch_size = 32
shuffle = True
prohibited_node_index=[],
max_null_iter=100
T = 200
num_sample = 1000
gamma = 0.
data_generator = dl.generators.MarkovBinaryDynamicsGenerator(graph_generator, state_generator,
                                                             batch_size,
                                                             shuffle=shuffle, 
                                                             prohibited_node_index=prohibited_node_index,
                                                             max_null_iter=max_null_iter)

# Model parameters
n_hidden = [32, 32]
n_heads = 6
wd = 1e-4
dropout = 0.
model = dl.models.GATMarkovBinaryPredictor(N,
                                           n_hidden,
                                           n_heads,
                                           weight_decay=wd,
                                           dropout=dropout)

# Trainer parameters
loss = keras.losses.binary_crossentropy
optimizer = keras.optimizers.Adam
metrics = ['accuracy']
learning_rate=1e-4
callbacks = []
np_seed = int(time())
tf_seed = int(time() + 10)
trainer = dl.models.Trainer(model, data_generator,
                            loss=loss,
                            optimizer=optimizer,
                            metrics=metrics,
                            learning_rate=learning_rate,
                            callbacks=callbacks,
                            numpy_seed=np_seed,
                            tensorflow_seed=tf_seed)



In [3]:
# Generating data for training
trainer.generate_data(num_sample, T, gamma=gamma, progress_bar=tqdm.tqdm_notebook)

In [4]:
# Training model

trainer.train(10, 100, verbose=1)

Epoch 1/10
100/100 [==============================] - 3s 32ms/step - loss: 0.9285 - acc: 0.6115
Epoch 2/10
100/100 [==============

In [5]:
import h5py
model_file = h5py.File('experiment_0_model.h5', 'w')

trainer.save_hdf5_model(model_file)
trainer.save_hdf5_optimizer(model_file)
trainer.save_hdf5_history(model_file)

model_file.close()